In [1]:
import pandas as pd
import regex
from pathlib import Path

### Load Data

In [2]:
"""
Get all Files and concatenate
"""
# Path to data files (might change)
parent_path = Path(".", "src", "movies", "movies").absolute()

files_path: list[Path] = []
for data_path in parent_path.iterdir():
    if regex.match(".+/film_info_part_\d.csv$", str(data_path)) != None:
        files_path.append(data_path)

# read all datafiles and concatenate in pandas
dfs = []
lengths = []
for file in files_path:
    df = pd.read_csv(file)
    lengths.append(len(df))
    dfs.append(df)

dataframe = pd.concat(dfs, ignore_index=True)
dataframe

,title,year,infobox,critics_count,critics_score,audience_count,audience_score,suppliers_list,rottentomatoes_year,rottentomatoes_genre,rottentomatoes_length
0,711 Ocean Drive,1950,"{'Directed by': ['Joseph M. Newman', '', ''], ...",3 Reviews,NaN,100+ Ratings,61.0,"[('vudu', 'Rent/buy'), ('amazon-prime-video-us...",1950,Crime/Drama,1h 42m
1,Abbott and Costello in the Foreign Legion,1950,"{'Directed by': ['Charles Lamont', '', ''], 'W...",3 Reviews,NaN,500+ Ratings,59.0,NaN,1950,Comedy,1h 20m
2,Ambush,1950,"{'Directed by': ['Sam Wood', '', ''], 'Screenp...",3 Reviews,NaN,Fewer than 50 Ratings,62.0,NaN,1949,Western,1h 29m
3,Annie Get Your Gun,1950,"{'Directed by': [' ', ' ', 'George Sidney', 'B...",12 Reviews,100.0,"5,000+ Ratings",67.0,NaN,1950,Musical,1h 47m
4,The Asphalt Jungle,1950,"{'Directed by': ['John Huston', '', ''], 'Scre...",35 Reviews,97.0,"5,000+ Ratings",87.0,"[('vudu', 'Rent/buy'), ('amazon-prime-video-us...",1950,Crime/Drama,1h 52m
...,...,...,...,...,...,...,...,...,...,...,...
12818,The Assassin,2015,"{'': ['Cìkè Niè Yǐnniáng', '', ''], 'Directed ...",128 Reviews,80.0,"5,000+ Ratings",48.0,"[('vudu', 'Rent/buy'), ('amazon-prime-video-us...",2015,Action/Drama,1h 45m
12819,Go Away Mr. Tumor,2015,"{'Directed by': ['Han Yan', '', ''], 'Starring...",1 Reviews,NaN,250+ Ratings,80.0,"[('amazon-prime-video-us', 'Rent/buy'), ('itun...",2015,Comedy,1h 25m
12820,"Love, At First…",2015,"{'Directed by': ['Tao Hai', '', ''], 'Starring...",0 Reviews,NaN,0 Ratings,NaN,"[('itunes', 'Rent/buy')]",2015,Romance,1h 36m
12821,A Tale of Three Cities,2015,"{'Directed by': ['Mabel Cheung', '', ''], 'Wri...",7 Reviews,43.0,0 Ratings,NaN,NaN,2015,Drama/Foreign,2h 10m


### Filter titles

In [3]:
# There shouldn't be any losses here
dataframe = dataframe.loc[~(dataframe["title"].isnull() | dataframe["title"].isna())]

In [4]:
dataframe.loc[12818]["infobox"]

"{'': ['Cìkè Niè Yǐnniáng', '', ''], 'Directed by': ['Hou Hsiao-hsien', '', ''], 'Written by': ['Hou Hsiao-hsien', 'Hsieh Hai-Meng', 'Chu T’ien-wen', 'Ah Cheng', '\\n \\n', ''], 'Produced by': ['Wen-Ying Huang', 'Liao Ching-Sung', '\\n \\n', ''], 'Starring': ['Shu Qi', 'Chang Chen', 'Zhou Yun', 'Satoshi Tsumabuki', '\\n \\n', ''], 'Cinematography': ['Mark Lee Ping Bin', '', ''], 'Edited by': ['Huang Chih-Chia', '', ''], 'Music by': ['Lim Giong', '', ''], 'Production companies': ['', ''], 'Distributed by': ['Well Go USA', 'StudioCanal', '\\n \\n', ''], 'Release dates': ['21\\xa0May\\xa02015', ' (', ')', '27\\xa0August\\xa02015', ' (China & Hong Kong)', '28\\xa0August\\xa02015', ' (Taiwan)', 'Cannes', '\\n \\n', ''], 'Running time': ['105 minutes', '', ''], 'Countries': ['Taiwan', 'China', 'Hong Kong', '\\n \\n', ''], 'Language': ['Mandarin', '', ''], 'Budget': ['90 million (', '14.9 million)', 'CN¥', 'US$', '', ''], 'Box office': [' (China)', ' (worldwide)', '\\n \\n', '']}"

### Filter years

In [5]:
"""
There might occur some differences between the Release Date on Wikipedia and Rotten Tomatoes possibly indicating a different movie alltogether
We're gonna simply remove the ones having a difference bigger than 1 year
"""
_filter = abs(dataframe["year"] - dataframe["rottentomatoes_year"]) > 1
different_movies = dataframe.loc[_filter]

print(f"removed {len(different_movies)} titles")
dataframe = dataframe.loc[~_filter]
dataframe

removed 1460 titles


,title,year,infobox,critics_count,critics_score,audience_count,audience_score,suppliers_list,rottentomatoes_year,rottentomatoes_genre,rottentomatoes_length
0,711 Ocean Drive,1950,"{'Directed by': ['Joseph M. Newman', '', ''], ...",3 Reviews,NaN,100+ Ratings,61.0,"[('vudu', 'Rent/buy'), ('amazon-prime-video-us...",1950,Crime/Drama,1h 42m
1,Abbott and Costello in the Foreign Legion,1950,"{'Directed by': ['Charles Lamont', '', ''], 'W...",3 Reviews,NaN,500+ Ratings,59.0,NaN,1950,Comedy,1h 20m
2,Ambush,1950,"{'Directed by': ['Sam Wood', '', ''], 'Screenp...",3 Reviews,NaN,Fewer than 50 Ratings,62.0,NaN,1949,Western,1h 29m
3,Annie Get Your Gun,1950,"{'Directed by': [' ', ' ', 'George Sidney', 'B...",12 Reviews,100.0,"5,000+ Ratings",67.0,NaN,1950,Musical,1h 47m
4,The Asphalt Jungle,1950,"{'Directed by': ['John Huston', '', ''], 'Scre...",35 Reviews,97.0,"5,000+ Ratings",87.0,"[('vudu', 'Rent/buy'), ('amazon-prime-video-us...",1950,Crime/Drama,1h 52m
...,...,...,...,...,...,...,...,...,...,...,...
12818,The Assassin,2015,"{'': ['Cìkè Niè Yǐnniáng', '', ''], 'Directed ...",128 Reviews,80.0,"5,000+ Ratings",48.0,"[('vudu', 'Rent/buy'), ('amazon-prime-video-us...",2015,Action/Drama,1h 45m
12819,Go Away Mr. Tumor,2015,"{'Directed by': ['Han Yan', '', ''], 'Starring...",1 Reviews,NaN,250+ Ratings,80.0,"[('amazon-prime-video-us', 'Rent/buy'), ('itun...",2015,Comedy,1h 25m
12820,"Love, At First…",2015,"{'Directed by': ['Tao Hai', '', ''], 'Starring...",0 Reviews,NaN,0 Ratings,NaN,"[('itunes', 'Rent/buy')]",2015,Romance,1h 36m
12821,A Tale of Three Cities,2015,"{'Directed by': ['Mabel Cheung', '', ''], 'Wri...",7 Reviews,43.0,0 Ratings,NaN,NaN,2015,Drama/Foreign,2h 10m


### Filter length

In [6]:
"""
We might want to include the runtime and convert it into minutes only 
"""

_pattern = regex.compile("^(\d+)h (\d+)m$")
def movie_runtime_transformer(x):
    match = _pattern.match(x.strip())
    # convert into minutes only
    ret_val = None
    if match:
        ret_val = int(match.group(1))*60 + int(match.group(2))
    return ret_val

# replace original length
dataframe["rottentomatoes_length"] = dataframe["rottentomatoes_length"].transform(movie_runtime_transformer)

### Filter Review counts

In [7]:
_pattern = regex.compile("^\D*(\d+).*$")
def review_count_transformer(x: str):
    x = x.replace(",", "")  # integer are divided by ',' for every 3 digits
    match = _pattern.match(x.strip())
    # convert into number of reviews only
    ret_val = 0
    
    if match:
        ret_val = int(match.group(1))
    return ret_val

# replace original counts
dataframe["audience_count"] = dataframe["audience_count"].transform(review_count_transformer)
dataframe["critics_count"] = dataframe["critics_count"].transform(review_count_transformer)
dataframe

,title,year,infobox,critics_count,critics_score,audience_count,audience_score,suppliers_list,rottentomatoes_year,rottentomatoes_genre,rottentomatoes_length
0,711 Ocean Drive,1950,"{'Directed by': ['Joseph M. Newman', '', ''], ...",3,NaN,100,61.0,"[('vudu', 'Rent/buy'), ('amazon-prime-video-us...",1950,Crime/Drama,102.0
1,Abbott and Costello in the Foreign Legion,1950,"{'Directed by': ['Charles Lamont', '', ''], 'W...",3,NaN,500,59.0,NaN,1950,Comedy,80.0
2,Ambush,1950,"{'Directed by': ['Sam Wood', '', ''], 'Screenp...",3,NaN,50,62.0,NaN,1949,Western,89.0
3,Annie Get Your Gun,1950,"{'Directed by': [' ', ' ', 'George Sidney', 'B...",12,100.0,5000,67.0,NaN,1950,Musical,107.0
4,The Asphalt Jungle,1950,"{'Directed by': ['John Huston', '', ''], 'Scre...",35,97.0,5000,87.0,"[('vudu', 'Rent/buy'), ('amazon-prime-video-us...",1950,Crime/Drama,112.0
...,...,...,...,...,...,...,...,...,...,...,...
12818,The Assassin,2015,"{'': ['Cìkè Niè Yǐnniáng', '', ''], 'Directed ...",128,80.0,5000,48.0,"[('vudu', 'Rent/buy'), ('amazon-prime-video-us...",2015,Action/Drama,105.0
12819,Go Away Mr. Tumor,2015,"{'Directed by': ['Han Yan', '', ''], 'Starring...",1,NaN,250,80.0,"[('amazon-prime-video-us', 'Rent/buy'), ('itun...",2015,Comedy,85.0
12820,"Love, At First…",2015,"{'Directed by': ['Tao Hai', '', ''], 'Starring...",0,NaN,0,NaN,"[('itunes', 'Rent/buy')]",2015,Romance,96.0
12821,A Tale of Three Cities,2015,"{'Directed by': ['Mabel Cheung', '', ''], 'Wri...",7,43.0,0,NaN,NaN,2015,Drama/Foreign,130.0


### Filter Infobox

In [8]:
"""
We want to preserve as much information as possible
The most important ones are budget, box office and release date (with month) which will have to be included
"""
import json

# Create pattern that ONLY changes " to ' when not needed for Names/language e.g.: "Edmond O'Brian"
_quote_mark_pattern = regex.compile("([A-Za-z][A-Za-z\s]+\"[A-Za-z]+)(?=[^\"]*\".*)")
_release_pattern = regex.compile(r"(\\xa0(?=\d+)(\d+))") # double enclosure group, potentially leading 0 
_structured_number_pattern = regex.compile('(?<=\".?)((\d+,?)+)(?=\")') # commas within natural numbers
_too_many_quotes_pattern = regex.compile('(\"{3,})')    # if 3 or more behind each other

def to_dict_transformer(x: str) -> dict | None:
    x = x.replace("'", "\"")    # needed for json
    # revert wanted '
    for p in _quote_mark_pattern.findall(x):
        x = x.replace(p, p.replace("\"", "'"))

    # replace broken numbers with correct ones
    # for broken, nums in _release_pattern.findall(x):
    #     x = x.replace(broken, f" {int(nums)}") 
    x = x.replace("\\xa0", " ")
    
    for broken, _ in _structured_number_pattern.findall(x):
        x = x.replace(broken, broken.replace(",", ""))

    for p in _too_many_quotes_pattern.findall(x):
        x = x.replace(p, '""')

    try:
        ret_val: dict = json.loads(x)
    except:
        ret_val = None  # most errors should now be fixed anyway
    return ret_val

# _str = 'ributed by": ["Loew"s, Inc.", "", ""], "Release date": ["J'
# print(_quote_mark_pattern.findall(_str))

"""
We can't use dataframe.transform here since then all the " and ' replacements are just reversed by panda
-> create new tags for each category at once
"""
dicts: list[dict[str, str] | None] = []
for dict_str in dataframe["infobox"]:
    dicts.append(to_dict_transformer(dict_str))


# Pattern that recognizes plural (s at end, not ies) and removes it
# words shorter than 3 letters are not plurals
_plural_pattern = regex.compile("(?<!i)(?<=[A-za-z]{3,})s(?=_?)(?![A-za-z])")

synonyms_dict = {}
local_path = Path(".", "src", "movies", "filter").absolute()
with open(Path(local_path, "synonyms.json")) as syns:
    synonyms_list: list[dict] = json.load(syns)
    # pack into one dict
    for _dic in synonyms_list:
        for _key, _list in _dic.items():
            synonyms_dict[_key] = _list

# create reverse lookup table
reverse_synonym_dict = {item: _key for _key, _list in synonyms_dict.items() for item in _list}

def replace_synonyms(in_str: str) -> str:
    base_synonym = reverse_synonym_dict.get(in_str)
    return base_synonym if base_synonym else in_str

def normalize_keys(in_str: str) -> str:
    # basic compression
    in_str = in_str.lower().strip().replace(" ", "_")
    # somtimes numbers are somehow included which we're gonna remove to get a smaller set
    # we're also gonna remove enclosing brackets around a letter
    in_str = regex.sub("\d|\(|\)", "", in_str)
    # some tags are given are in plural some in singular. We're gonna reduce that to always singular by 
    # simply assuming that only plurals end with 's'ArithmeticError
    in_str = regex.sub("ies", "y", in_str)
    # apply regex based plural filter
    in_str = _plural_pattern.sub("", in_str)

    return in_str

category_counter: dict[str, int] = dict()
# normalize categories
for dic in dicts:
    if dic != None:
        key_copy = set(dic.keys())
        for c_key in key_copy:
            normalized_key = normalize_keys(c_key)
            
            # replace synonyms
            normalized_key = replace_synonyms(normalized_key)

            dic[normalized_key] = dic.pop(c_key)

            # add new entry or increase counter
            if category_counter.get(normalized_key):
                category_counter[normalized_key] += 1
            else:
                category_counter[normalized_key] = 1

print(category_counter)

{'directed_by': 11017, 'edited_by': 9999, 'music_by': 9935, 'box_office': 6745, 'language': 10752, 'budget': 5260, 'distributed_by': 9799, 'color_proces': 390, 'running_time': 10696, 'production_company': 7861, 'release_date': 10855, 'starring': 10776, 'country': 10686, 'produced_by': 10374, 'cinematography': 10164, 'written_by': 8525, 'narrated_by': 282, 'screenplay_by': 3772, 'based_on': 3528, '': 1377, 'released': 709, 'animation_by': 9, 'author': 32, 'recorded': 287, 'published': 7, 'illustrated_by': 5, 'original_title': 14, 'text': 1, 'published_by': 38, 'dialogue_by': 13, 'length': 509, 'original_air_date': 1, 'episode': 4, 'production_code': 2, 'place_premiered': 8, 'date_premiered': 9, 'genre': 113, 'original_language': 138, 'publication_date': 28, 'page': 18, 'literally': 86, 'original_network': 116, 'of_season': 2, 'country_of_origin': 131, 'of_episode': 6, 'executive_producer': 70, 'related': 3, 'picture_format': 54, 'created_by': 6, 'original_release': 134, 'audio_format': 

In [9]:
test = "'Release dates': ['21\\xa0May\\xa02015', ' (', ')', '27\\xa0August\\xa02015', ' (China & Hong Kong)', '28\\xa0August\\xa02015', ' (Taiwan)', 'Cannes', '\\n "
_release_pattern.findall(test)

[('\\xa02015', '2015'), ('\\xa02015', '2015'), ('\\xa02015', '2015')]

In [10]:
dataframe["infobox"]

0        {'Directed by': ['Joseph M. Newman', '', ''], ...
1        {'Directed by': ['Charles Lamont', '', ''], 'W...
2        {'Directed by': ['Sam Wood', '', ''], 'Screenp...
3        {'Directed by': [' ', ' ', 'George Sidney', 'B...
4        {'Directed by': ['John Huston', '', ''], 'Scre...
                               ...                        
12818    {'': ['Cìkè Niè Yǐnniáng', '', ''], 'Directed ...
12819    {'Directed by': ['Han Yan', '', ''], 'Starring...
12820    {'Directed by': ['Tao Hai', '', ''], 'Starring...
12821    {'Directed by': ['Mabel Cheung', '', ''], 'Wri...
12822    {'Directed by': ['Cao Baoping', '', ''], 'Scre...
Name: infobox, Length: 11363, dtype: object

### Convert Info Box into new Tags

In [11]:
"""
Split dictionary into single data types for easier referencing
"""
# how much percent of the tag with the most occurences another tag MUST have to be included in the dataset
INCLUSION_MARGIN = 1/3
MAX_COUNT = max(category_counter.values())
LOWER_BOUNDARY = int(INCLUSION_MARGIN * MAX_COUNT)

assert(len(dicts) == len(dataframe))

valid_tags: list[tuple[str, int]] = list(filter(lambda _tag_val_tup: _tag_val_tup[1] >= LOWER_BOUNDARY, category_counter.items()))
new_col_dict: dict[str, list] = {_key: [] for _key, _ in valid_tags}

for dic in dicts:
    for tag, _ in valid_tags:
        if dic: # could be None if not readable
            # get all rows with this tag and add as column to dataframe
            item: str | None = dic.get(tag)
            new_col_dict[tag].append(item)
        else:
            new_col_dict[tag].append(None)

# check for any lost items
for _tag, _list in new_col_dict.items():
    assert(len(_list) == len(dataframe))

# add columns to dataframe
existing_keys = list(dataframe.keys())
for _tag, _list in new_col_dict.items():
    # prevent overwriting
    if _tag in existing_keys:
        _tag += "_info"
    dataframe[_tag] = _list

In [12]:
dataframe.keys()

Index(['title', 'year', 'infobox', 'critics_count', 'critics_score',
       'audience_count', 'audience_score', 'suppliers_list',
       'rottentomatoes_year', 'rottentomatoes_genre', 'rottentomatoes_length',
       'directed_by', 'edited_by', 'music_by', 'box_office', 'language',
       'budget', 'distributed_by', 'running_time', 'production_company',
       'release_date', 'starring', 'country', 'produced_by', 'cinematography',
       'written_by', 'screenplay_by'],
      dtype='object')

### Reduce Lists within entries

In [13]:
dataframe.loc[0]

title                                                      711 Ocean Drive
year                                                                  1950
infobox                  {'Directed by': ['Joseph M. Newman', '', ''], ...
critics_count                                                            3
critics_score                                                          NaN
audience_count                                                         100
audience_score                                                        61.0
suppliers_list           [('vudu', 'Rent/buy'), ('amazon-prime-video-us...
rottentomatoes_year                                                   1950
rottentomatoes_genre                                           Crime/Drama
rottentomatoes_length                                                102.0
directed_by                                         [Joseph M. Newman, , ]
edited_by                                                [Bert Jordan, , ]
music_by                 

In [14]:
"""
Due to the data collection process, the infobox contains many empty elements
These need to be reduced and ideally into one element only
"""
def reduce_list(lis: list[str] | None) -> list:
    if lis:
        reduced_list = []
        for item in lis:
            item = item.replace("\n", "").strip().lower()
            if item:
                reduced_list.append(item)
        return reduced_list
    else:
        return None

# transform data
for _tag in new_col_dict.keys():
    dataframe[_tag] = dataframe[_tag].transform(reduce_list)
    # print(dataframe[_tag].transform(reduce_list))

### Convert Release dates

In [15]:
dataframe["release_date"]

0                          [july 1, 1950, (united states)]
1                                          [july 24, 1950]
2                                       [january 13, 1950]
3                                          [july 17, 1950]
4                          [may 12, 1950, (united states)]
                               ...                        
12818    [21 may 2015, (, ), 27 august 2015, (china & h...
12819                                    [august 13, 2015]
12820                                     [august 7, 2015]
12821    [27 august 2015, (china), 3 september 2015, (h...
12822    [june 15, 2015, (, ), august 27, 2015, (china)...
Name: release_date, Length: 11363, dtype: object

In [16]:
"""
There are some different variants for writing the date unfortunately so we will use regex to extrapolate month day and year and bring it into the following format:
<yyyy>-<mm>-<dd>
furthermore, there are different release dates. We're only gonna look at the oldest release (which came first)
"""
from dateutil.parser import *

# test = dataframe["release_date"][12818]
# print(str(parse(test[0]).date()))
# print(parse(test[0]) < parse("21.05.2016"))

def get_first_date(dates: list[str] | None) -> str | None:
    if dates:
        parsed_dates = []
        for line in dates:
            try:
                _date = parse(line)
                parsed_dates.append(_date)
            except:
                # unrecognizable date or simply something different (e.g. countries)
                continue
        # get oldest date as str
        return str(min(parsed_dates).date()) if parsed_dates else None
    return None

dataframe["release_date"] = dataframe["release_date"].transform(get_first_date)

# recalculate index
dataframe = dataframe.reset_index()

In [17]:
# amount of valid dates
len(dataframe["release_date"].loc[~dataframe["release_date"].isnull()])

10557

### Box Office/Budget Filter

In [18]:
cols = [k for k in new_col_dict.keys()]
dataframe[cols]

,directed_by,edited_by,music_by,box_office,language,budget,distributed_by,running_time,production_company,release_date,starring,country,produced_by,cinematography,written_by,screenplay_by
0,[joseph m. newman],[bert jordan],[sol kaplan],[$1550000],[english],[$300000],[columbia pictures],[102 minutes],[frank seltzer productions],1950-07-01,"[edmond o'brien, joanne dru, otto kruger]",[united states],[frank n. seltzer],[franz planer],"[richard english, francis swann]",None
1,[charles lamont],[frank gross],None,[$1250000],[english],[$679687],[universal-international],[80 minutes],[universal-international],1950-07-24,"[bud abbott, lou costello, patricia medina, wa...",[united states],[robert arthur],[george robinson],"[john grant, martin ragaway, leonard stern]",None
2,[sam wood],[ben lewis],[rudolph g. kopp],[$3215000],[english],[$1754000],[metro-goldwyn-mayer],[89 minutes],[metro-goldwyn-mayer],1950-01-13,"[robert taylor, john hodiak, arlene dahl]",[united states],"[, sam wood, armand deutsch]",[harold lipstein],None,[marguerite roberts]
3,"[george sidney, busby berkeley, charles walters]",[james e. newcom],"[songs: (lyrics and music by), music direction...",[$7756000],[english],[$3734000],"[loew's, inc.]",[107 minutes],[metro-goldwyn-mayer],1950-07-17,"[betty hutton, howard keel, louis calhern, kee...",[united states],"[arthur freed, roger edens]",[charles rosher],None,[sidney sheldon]
4,[john huston],[george boemler],[miklós rózsa],None,[english],[$1232000],[loew's inc.],[112 minutes],[metro-goldwyn-mayer],1950-05-12,"[sterling hayden, louis calhern, jean hagen]",[united states],[arthur hornblow jr.],[harold rosson],None,"[john huston, ben maddow]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358,[hou hsiao-hsien],[huang chih-chia],[lim giong],"[(china), (worldwide)]",[mandarin],"[90 million (, 14.9 million), cn¥, us$]","[well go usa, studiocanal]",[105 minutes],[],2015-05-21,"[shu qi, chang chen, zhou yun, satoshi tsumabuki]","[taiwan, china, hong kong]","[wen-ying huang, liao ching-sung]",[mark lee ping bin],"[hou hsiao-hsien, hsieh hai-meng, chu t’ien-we...",None
11359,[han yan],None,None,[us$80.12 million],[mandarin],None,[china lion (u.s.)],[128 minutes],[wanda pictures],2015-08-13,"[bai baihe, daniel wu]",[china],None,None,None,None
11360,[tao hai],None,None,[],[mandarin],None,"[china film, eastern mordor, tianjing yinhe me...",[98 minutes],[shanghai jinse tianxia entertainment beijing ...,2015-08-07,"[juck zhang, xiaofeng li, yao zhang, jingjing ...",[china],None,None,None,None
11361,[mabel cheung],None,None,None,[mandarin],None,None,[130 minutes],None,2015-08-27,"[huang jue, li jianyi, sean lau, tang wei, qin...","[china, hong kong]",None,None,"[mabel cheung, alex law]",None


In [19]:
inflation_table_US = pd.read_csv(Path(local_path, "inflation_rate_usa.csv"), dtype=float)
inflation_table_US["Year"] = inflation_table_US["Year"].transform(lambda x: int(x))
# TODO use IMF Table for different countries
FIRST_YEAR_US = 1913
LAST_YEAR_US = 2022
MONTH_MAPPING = {
    1: "Jan",
    2: "Feb",
    3: "Mar",
    4: "Apr",
    5: "May",
    6: "June",
    7: "July",
    8: "Aug",
    9: "Sep",
   10: "Oct",
   11: "Nov",
   12: "Dec",
}
"""
Inflation Table contains CPI in Columns Jan-Dec and Avg, Dec-Dec and Avg-Avg is already given in percent
"""

def inflation_calculator_US(value: float, from_year: int, to_year: int = 2022, from_month: int = None, to_month: int = None) -> float:
    """
    Calculate Inflation per year or even per month if <from_month> and <to_month> are specified
    Inflation can be efficiently calculated with CPI money*(<to_cpi> / <from_cpi>)
    Rounded to 2 decimals
    """
    if from_year < FIRST_YEAR_US or to_year > LAST_YEAR_US:
        if to_year < FIRST_YEAR_US or from_year > LAST_YEAR_US:
            raise ValueError("years are out of range of dataset")
    if from_month:
        if from_month < 1 or from_month > 12:
            raise ValueError("from months are out of range")
    if to_month:
        if to_month < 1 or to_month > 12:
            raise ValueError("to months are out of range")

    _from_table = inflation_table_US.loc[from_year-FIRST_YEAR_US]
    _to_table = inflation_table_US.loc[to_year-FIRST_YEAR_US]

    _from_pci: float = _from_table[MONTH_MAPPING[from_month]] if from_month else _from_table["Avg"]
    _to_pci: float = _to_table[MONTH_MAPPING[to_month]] if to_month else _to_table["Avg"]

    return round(value * (_to_pci / _from_pci), 2)

# inflation_calculator_US(1.0, 1913, 1917, 1, 12)

In [20]:
CURRENCY_CPI_MAPPING: dict[str: str] = {
    "US_DOLLAR" : "United States",
    "NZ_DOLLAR" : "New Zealand",
    "AUS_DOLLAR" : "Australia",
    "HK_DOLLAR" : "China, P.R.: Hong Kong",
    "NT_DOLLAR" : "China, P.R.: Hong Kong", # "Taiwan" is not in the Database assume same growth as Hong Kong
    "MX_DOLLAR" : "Mexico",
    "GB_POUND" : "United Kingdom",
    "EURO" : "Euro Area",
    "JP_YEN" : "Japan",
    "CN_YUAN" : "China, P.R.: Mainland",
    "RUBLES" : "Russian Federation",
    "SWED_KRON" : "Sweden",
    "INDIAN_RUPEE" : "India",
    "TURK_LIRA" : "Türkiye, Rep of",
    "MALAYSIA_RM" : "Malaysia",
}
inflation_table = pd.read_csv(Path(local_path, "CPI_timeSeries", "CPI_simple.csv"))

# TODO use IMF Table for different countries
FIRST_YEAR = 1950
LAST_YEAR = 2022

# TODO inflation calculator for different countries and conversion Rate!!

def inflation_calculator(currency: str, value: float, from_year: int, to_year: int = 2022, from_month: int = None, to_month: int = None) -> float:
    """
    Calculate Inflation per year or even per month if <from_month> and <to_month> are specified
    Inflation can be efficiently calculated with CPI money*(<to_cpi> / <from_cpi>)
    Rounded to 2 decimals.

    @param currency must be part of the translation_dict_CPI mapping!
    
    It is recommended to calculate up to 2017, and the convert to US $ since the exchange database wasn't updated anymore since may 2018
    """
    if from_year < FIRST_YEAR or to_year > LAST_YEAR:
        if to_year < FIRST_YEAR or from_year > LAST_YEAR:
            raise ValueError("years are out of range of dataset")
    if from_month:
        if from_month < 1 or from_month > 12:
            raise ValueError("from months are out of range")
    if to_month:
        if to_month < 1 or to_month > 12:
            raise ValueError("to months are out of range")

    _from_index = f"{from_year}" + (f"M{from_month}" if from_month else "")
    _to_index = f"{to_year}" + (f"M{to_month}" if to_month else "")

    country_inflation_table = inflation_table.loc[inflation_table["Country Name"] == CURRENCY_CPI_MAPPING[currency]].reset_index()

    _from_pci = country_inflation_table[_from_index][0]
    _to_pci = country_inflation_table[_to_index][0]

    return round(value * (_to_pci / _from_pci), 2)

In [21]:
print(inflation_calculator("US_DOLLAR", 1.0, 1950, 1980, 1, 12))
print(inflation_calculator_US(1.0, 1950, 1980, 1, 12))
# Close enough

3.49
3.67


In [22]:
CURRENCY_EXCHANGE_MAPPING: dict[str: str|None] = {
    "US_DOLLAR" : "U.S. Dollar",
    "NZ_DOLLAR" : "New Zealand Dollar",
    "AUS_DOLLAR" : "Australian Dollar",
    "HK_DOLLAR" : None,   # Hong Kong is not in the Database
    "NT_DOLLAR" : None,   # Taiwan is not in the Database
    "MX_DOLLAR" : "Mexican Peso",
    "GB_POUND" : "U.K. Pound Sterling",
    "EURO" : "Euro",
    "JP_YEN" : "Japanese Yen",
    "CN_YUAN" : "Chinese Yuan",
    "RUBLES" : "Russian Ruble",
    "SWED_KRON" : "Swedish Krona",
    "INDIAN_RUPEE" : "Indian Rupee",
    "TURK_LIRA" : None,    # Turkey is not in the Database
    "MALAYSIA_RM" : "Malaysian Ringgit",
}
exchange_table = pd.read_csv(Path(local_path, "exchange_rate", "currency_exchange_rates.csv"))
# take only one timestamp where No value is NaN
exchange_table = exchange_table.loc[exchange_table["Date"] == "2017-1-18"]

CURRENCY_EXCHANGE_VALUE_MAPPING: dict[str, float] = {
    base_key: float(exchange_table[ex_key]) for base_key, ex_key in CURRENCY_EXCHANGE_MAPPING.items() if ex_key
}
# add missing avg exchange rates for 2017, copied manually from https://www.exchangerates.org.uk
update_dict = {
    "HK_DOLLAR" : 7.7925,
    "NT_DOLLAR" : 30.4252,
    "TURK_LIRA" : 3.6462,
}
CURRENCY_EXCHANGE_VALUE_MAPPING.update(update_dict)

def convert_into_dollar(currency: str, value: float) -> float:
    """
    Takes a currency from CURRENCY_EXCHANGE_VALUE_MAPPING and converts it into dollar.
    This is ONLY somewhat accurate for January of 2017!!!
    """
    return value / CURRENCY_EXCHANGE_VALUE_MAPPING[currency]
    

### Combine Both functions to get money in US$ for 2022

In [23]:
def convert_value_to_recent_us_dollar(currency: str, value: float, from_year: int, from_month: int|None = None) -> float:
    """
    @param currency original currency of <value>
    @param value amount of original currency
    @param from_year how much inflation took place since the value was recent
    @param from_month which month in <from_year> 
    """
    exchangable_value = inflation_calculator(currency, value, from_year, 2017, from_month, 1)
    new_value = convert_into_dollar(currency, exchangable_value)
    return inflation_calculator("US_DOLLAR", new_value, 2017, 2022, 1, 12)

convert_value_to_recent_us_dollar("JP_YEN", 100, 1970, 1)
# CURRENCY_EXCHANGE_VALUE_MAPPING

3.48

In [24]:
"""
First we need to convert all written text into a natural number (e.g. 46.75 million = 46750000)
After that, the really difficult part starts by accounting for inflation and different currencies
We want US$ only at the end
"""

_number_pattern = regex.compile("(\d+)")
_reduced_mbillion_pattern = regex.compile("^((\d+\.?\d*)\s?(millions?|billions?|crores?))(?=.*)")
_mbillion_pattern = regex.compile("(?<=.*\s|\W)((\d+\.?\d*)\s?(millions?|billions?|crores?))(?=.*)")
# some entries already or only contain an equivalent for decades later than when the movei was released
# this needs to be regarded for the inflation adjustments
_equivalent_pattern = regex.compile(".*[\s\(]((\w?\w?\D)(\d+)\sin\s(\d{4})).*") # groups -> capture, currency, amount, year

"""
Predefined Pattern for the Value type of the return dict of money_heuristic
This is required for the inflation calculator to adjust the values more correctly
if one of the ATTRIBUTES items is None, use the original entry data to infer the used values
"""
CURRENCY = str|None
YEAR = int|None
MONEY_ATTRIBUTES = list[CURRENCY, YEAR]
AMOUNT_ATTRIBUTES = 2

NON_US_DOLLAR_SCORE = 6
# each tuple has Name of Currency/Country and associated score (loosley based on conversion factor)
# sorted with decreasing score!
_optional_value_pattern = "()"
# patterns first and last group are always the POTENTIAL number (maybe doesn't exist)
_currencies_patterns: list[tuple[str, "regex.Pattern", int]] = [
    ("US_DOLLAR", regex.compile("(\d+)?\s?((us)?\$)\s?(\d+)?"), 10),    # score must only be applied if NONE of the other _DOLLAR patterns matched!
    ("NZ_DOLLAR", regex.compile("(\d+)?\s?((au?)\$)\s?(\d+)?"), NON_US_DOLLAR_SCORE),
    ("AUS_DOLLAR", regex.compile("(\d+)?\s?((nz)\$)\s?(\d+)?"), NON_US_DOLLAR_SCORE),
    ("HK_DOLLAR", regex.compile("(\d+)?\s?((hk)\$)\s?(\d+)?"), NON_US_DOLLAR_SCORE),
    ("NT_DOLLAR", regex.compile("(\d+)?\s?((nt)\$)\s?(\d+)?"), NON_US_DOLLAR_SCORE),
    ("MX_DOLLAR", regex.compile("(\d+)?\s?((mx)\$)\s?(\d+)?"), NON_US_DOLLAR_SCORE),
    ("GB_POUND", regex.compile("(\d+)?\s?(£)\s?(\d+)?"), 5),
    ("EURO", regex.compile("(\d+)?\s?(€)\s?(\d+)?"), 5),
    ("JP_YEN", regex.compile("(\d+)?\s?((?<!cn)¥)\s?(\d+)?"), 3),
    ("CN_YUAN", regex.compile("(\d+)?\s?(cn¥|yuan|rmb)\s?(\d+)?"), 3),
    ("RUBLES", regex.compile("(\d+)?\s?(rub|₽)\s?(\d+)?"), 3),
    ("SWED_KRON", regex.compile("(\d+)?\s?(sek)\s?(\d+)?"), 3),
    ("INDIAN_RUPEE", regex.compile("(\d+)?\s?(₹)\s?(\d+)?"), 3),
    ("TURK_LIRA", regex.compile("(\d+)?\s?(₺)\s?(\d+)?"), 3),
    ("MALAYSIA_RM", regex.compile("(\d+)?\s?(rm)\s?(\d+)?"), 3),
]

def money_heuristic(entries: list[str]) -> tuple[int, MONEY_ATTRIBUTES] | None:
    """
    wikipedia often has multiple entries about the box office (global, within the country, other currencies, etc.)
    This function assigns a scoring system for these entries to estimate the most beneficial for our dataset.
    
    @return the most useful entry based on heuristics, condensed on number only, additional ATTRIBUTES needed for conversion/inflation
    """

    def multiplier(selector: str) -> float:
        """
        the equivalent in numbers to a word
        """
        return {
            "million": 10**6,
            "millions": 10**6,
            "crore": 10**7, # east asian for 10 million, also indian currency
            "crores": 10**7,
            "billion": 10**9,
            "billions": 10**9,
        }[selector]
    
    
    def amount_heuristic(_heuristics: dict[int, int], _entries: list[str]) -> None:
        """
        modifies heuristic based on number 
        """
        numbers: list[int] = []
        # get biggest number in one entry
        for index, entry in enumerate(_entries):
            numbers.append(-1)
            for num in _number_pattern.findall(entry):
                int_num = int(num)
                if int_num > numbers[-1]:
                    numbers[-1] = int_num

        # get heuristic value based on size
        argmax_list = [0 for i in range(len(_entries))]
        for index, num in enumerate(numbers):
            for i in range(len(_entries)):
                argmax_list[i] += 1 if numbers[i] > num else 0

        # apply scores
        for index, points in enumerate(argmax_list):
            _heuristics[index] += points
    

    def currency_heuristic(_heuristics: dict[int, int], _entries: list[str]) -> dict[int, MONEY_ATTRIBUTES]:
        """
        This method tries to find out which currencies are probably used in each entry.
        We want us$ if possible since we only need to account for inflation then and not for exchange factor as well
        """
        curr_heur: dict[int, int] = {}
        _attributes: dict[int, MONEY_ATTRIBUTES] = {} 
        for index, entry in enumerate(_entries):
            curr_heur[index] = 0    # initiate default score of +0
            _attributes[index] = [None for _ in range(AMOUNT_ATTRIBUTES)]   # initiate empty attribute list

            for cur_name, pattern, score in _currencies_patterns[:0:-1]: # use inverse list and omit US$
                if pattern.findall(entry):
                    curr_heur[index] = score
                    _attributes[index][0] = cur_name
            # check if other form of dollar was already found
            cur_name, pattern, score = _currencies_patterns[0]
            if curr_heur[index] != NON_US_DOLLAR_SCORE and pattern.findall(entry):
                curr_heur[index] = score
                _attributes[index][0] = cur_name    # US_DOLLAR

        # apply scores
        for index, points in curr_heur.items():
            _heuristics[index] += points
        return _attributes


    def equivalency_heuristic(_heuristics: dict[int, int], _entries: list[str], _attributes: dict[int, MONEY_ATTRIBUTES]) -> None:
        """
        Some entries already include conversions for a more recent timestamp than when the movie originally released.
        We want to utilize this extra information since this is more often more accurate (somtimes even the only) information.
        We give an additional +5 score for more recent information.

        This method additionally also filters out the pure number for equivalency terms only
        """
        for index, entry in enumerate(_entries):
            for _capture, currency, amount, year in _equivalent_pattern.findall(entry):
                _heuristics[index] += 5
                _attributes[index][1] = int(year)    # get date of already converted money
                # get currency of converted money
                for cur_name, pattern, _ in _currencies_patterns[::-1]:
                    if pattern.findall(currency):
                        _attributes[index][0] = cur_name
                        _entries[index] = amount
                        break   # don't check for multiple possibilites


    def pure_number_filter(_entries: list[str], _attributes: dict[int, MONEY_ATTRIBUTES]) -> list[int]:
        """
        Get number with associated currency (_entries MUST have run through currency_heuristic!)

        @return list with number only, type of currency is stored in attributes
        """
        def get_max_number(_entry: str) -> int:
            max: int = 0
            for str_num in _number_pattern.findall(_entry):
                num = int(str_num)  # shouldn't raise exceptions since guarded by regex
                if num > max:
                    max = num
            return max
        
        ret_entries: list[int] = _entries.copy()
        for index, entry in enumerate(_entries):
            pattern = None
            # get correct pattern (no dictionary used because of undeterministic order within the data structure)
            for cur_name, _pattern, _ in _currencies_patterns:
                if _attributes[index][0] == cur_name:
                    pattern = _pattern
                    break
            if pattern:
                # found pattern to currency, use first and last capture group to get amount
                for capture_groups in pattern.findall(entry):
                    if capture_groups[-1]:  # is empty if no number behind currency (is most likely the case)
                        ret_entries[index] = int(capture_groups[-1])
                    elif capture_groups[0]:  # is empty if no number before currency (happens somtimes)
                        ret_entries[index] = int(capture_groups[0])
                    else:   # Can happen due to equivalency_heuristic. should not happen otherwise
                        ret_entries[index] = get_max_number(entry)
            else:
                # unidentified currency, just get highest number, currency will be inferred by country of origin (or rather just Dollar?)
                ret_entries[index] = get_max_number(entry)
        return ret_entries


    # filter unwanted characters ',' (in numbers)
    filtered_entries: list[str] = [item.replace(",", "").strip().lower() for item in entries]

    # we assume a box office to include numbers in their string (could also be year, but most of the times)
    # if there isn't a number, concatenate the string with the entry left to it (or right in case its the first index) 
    to_keep: list[int] = []
    for index, entry in enumerate(filtered_entries):
        if not _number_pattern.findall(entry):
            if (index-1) not in to_keep and index+1 < len(filtered_entries):
                filtered_entries[index+1] = f"{entry} {filtered_entries[index+1]}"
            elif to_keep:
                filtered_entries[to_keep[-1]] += f" {entry}"
        else:
            to_keep.append(index)
    # remove entry that was concatenated
    _filtered_entries = []
    for keep in to_keep:
        _filtered_entries.append(filtered_entries[keep])
    del filtered_entries
    filtered_entries = _filtered_entries

    # convert 'million' 'billion' 'crore' into numbers
    for index, entry in enumerate(filtered_entries):
        # print(entry)
        filtered_entries[index] = entry
        # try this pattern first in case string starts directly with number
        for capture, _float, mbil in _reduced_mbillion_pattern.findall(entry):
            replace_with = str(int(float(_float) * multiplier(mbil)))
            # crore is special case since its also the currency itself and not only a multiplier
            if "crore" in capture:
                # indian rupees
                replace_with = f"₹{replace_with}"
            filtered_entries[index] = filtered_entries[index].replace(capture, replace_with)
        # try the extended pattern
        for capture, _float, mbil in _mbillion_pattern.findall(filtered_entries[index]):
            replace_with = str(int(float(_float) * multiplier(mbil)))
            # crore is special case since its also the currency itself and not only a multiplier
            if "crore" in capture:
                # indian rupees
                replace_with = f"₹{replace_with}"
            filtered_entries[index] = filtered_entries[index].replace(capture, replace_with)
    
    if filtered_entries:
        # apply heuristics, select best candidate
        # selection based on index
        heuristics: dict[int, int] = {_ind: 1 for _ind in range(len(filtered_entries))}
        
        amount_heuristic(heuristics, filtered_entries)
        attributes: dict[int, MONEY_ATTRIBUTES] = currency_heuristic(heuristics, filtered_entries)
        equivalency_heuristic(heuristics, filtered_entries, attributes)

        # get pure numbers only
        filtered_entries: list[int] = pure_number_filter(filtered_entries, attributes)
        
        # get entry with highest heuristic score
        _argmax = -1
        max_score = -1
        for index, score in heuristics.items():
            if score > max_score:
                max_score = score
                _argmax = index
        return filtered_entries[_argmax], attributes[_argmax]
    return None


In [25]:
# Local Tests
test_number = "asddhasdha asdhasiodhausdh 321 dasdhadsad"
test_number2 = "a da 123123123 asd 123"
print(_number_pattern.findall(test_number))
print(_number_pattern.findall(test_number2))

test_mb = "da d, $3.12 million das 5.21 billions"
#_mbillion_pattern.match(test_mb)
print(_reduced_mbillion_pattern.findall(test_mb))
print(_mbillion_pattern.findall(test_mb))

test_ep = "(us$26000000 in 2020"   # "d asd asdd ( us$26000000 in 2020"
print(_equivalent_pattern.findall(test_ep))

test_currency = "¥" # "cn¥" should not be found
print(_currencies_patterns[8][1].findall(test_currency))

money_heuristic(["da", "d, ", " 3.12 million das 5.21 billions", "$5.12 million", "3 crore", "to $4 million in 2020", "hk$6,384,789"])

['321']
['123123123', '123']
[]
[('3.12 million', '3.12', 'million'), ('5.21 billions', '5.21', 'billions')]
[('us$26000000 in 2020', 'us$', '26000000', '2020')]
[('', '¥', '')]


('4000000', ['US_DOLLAR', 2020])

In [26]:
# Apply heuristic selection and inflation and conversion to database

def transform_money(money_tag: str) -> "pd.DataFrame":
    box_off = dataframe[money_tag].copy()
    for key, entries in dataframe[money_tag].items():
        final_value = None
        if entries:
            val_att = money_heuristic(entries)
            if val_att:
                value: float = float(val_att[0])
                currency, year = val_att[1]   # list[CURRENCY, YEAR]
                # for simplicity, if no currency is given, we will just use dollar
                if not currency:
                    currency = "US_DOLLAR"
                if not year:
                    year = dataframe[key:key+1]["year"][key]
                date_str = dataframe[key:key+1]["release_date"][key]
                month = None
                if date_str:
                    month = parse(date_str).month
                
                final_value = convert_value_to_recent_us_dollar(currency, value, year, month)
        box_off[key:key+1][key] = final_value
    return box_off


dataframe["box_office"] = transform_money("box_office")
dataframe["budget"] = transform_money("budget")

### All monetary Values are now in dollar 

In [27]:
dataframe

,index,title,year,infobox,critics_count,critics_score,audience_count,audience_score,suppliers_list,rottentomatoes_year,...,distributed_by,running_time,production_company,release_date,starring,country,produced_by,cinematography,written_by,screenplay_by
0,0,711 Ocean Drive,1950,"{'Directed by': ['Joseph M. Newman', '', ''], ...",3,NaN,100,61.0,"[('vudu', 'Rent/buy'), ('amazon-prime-video-us...",1950,...,[columbia pictures],[102 minutes],[frank seltzer productions],1950-07-01,"[edmond o'brien, joanne dru, otto kruger]",[united states],[frank n. seltzer],[franz planer],"[richard english, francis swann]",None
1,1,Abbott and Costello in the Foreign Legion,1950,"{'Directed by': ['Charles Lamont', '', ''], 'W...",3,NaN,500,59.0,NaN,1950,...,[universal-international],[80 minutes],[universal-international],1950-07-24,"[bud abbott, lou costello, patricia medina, wa...",[united states],[robert arthur],[george robinson],"[john grant, martin ragaway, leonard stern]",None
2,2,Ambush,1950,"{'Directed by': ['Sam Wood', '', ''], 'Screenp...",3,NaN,50,62.0,NaN,1949,...,[metro-goldwyn-mayer],[89 minutes],[metro-goldwyn-mayer],1950-01-13,"[robert taylor, john hodiak, arlene dahl]",[united states],"[, sam wood, armand deutsch]",[harold lipstein],None,[marguerite roberts]
3,3,Annie Get Your Gun,1950,"{'Directed by': [' ', ' ', 'George Sidney', 'B...",12,100.0,5000,67.0,NaN,1950,...,"[loew's, inc.]",[107 minutes],[metro-goldwyn-mayer],1950-07-17,"[betty hutton, howard keel, louis calhern, kee...",[united states],"[arthur freed, roger edens]",[charles rosher],None,[sidney sheldon]
4,4,The Asphalt Jungle,1950,"{'Directed by': ['John Huston', '', ''], 'Scre...",35,97.0,5000,87.0,"[('vudu', 'Rent/buy'), ('amazon-prime-video-us...",1950,...,[loew's inc.],[112 minutes],[metro-goldwyn-mayer],1950-05-12,"[sterling hayden, louis calhern, jean hagen]",[united states],[arthur hornblow jr.],[harold rosson],None,"[john huston, ben maddow]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358,12818,The Assassin,2015,"{'': ['Cìkè Niè Yǐnniáng', '', ''], 'Directed ...",128,80.0,5000,48.0,"[('vudu', 'Rent/buy'), ('amazon-prime-video-us...",2015,...,"[well go usa, studiocanal]",[105 minutes],[],2015-05-21,"[shu qi, chang chen, zhou yun, satoshi tsumabuki]","[taiwan, china, hong kong]","[wen-ying huang, liao ching-sung]",[mark lee ping bin],"[hou hsiao-hsien, hsieh hai-meng, chu t’ien-we...",None
11359,12819,Go Away Mr. Tumor,2015,"{'Directed by': ['Han Yan', '', ''], 'Starring...",1,NaN,250,80.0,"[('amazon-prime-video-us', 'Rent/buy'), ('itun...",2015,...,[china lion (u.s.)],[128 minutes],[wanda pictures],2015-08-13,"[bai baihe, daniel wu]",[china],None,None,None,None
11360,12820,"Love, At First…",2015,"{'Directed by': ['Tao Hai', '', ''], 'Starring...",0,NaN,0,NaN,"[('itunes', 'Rent/buy')]",2015,...,"[china film, eastern mordor, tianjing yinhe me...",[98 minutes],[shanghai jinse tianxia entertainment beijing ...,2015-08-07,"[juck zhang, xiaofeng li, yao zhang, jingjing ...",[china],None,None,None,None
11361,12821,A Tale of Three Cities,2015,"{'Directed by': ['Mabel Cheung', '', ''], 'Wri...",7,43.0,0,NaN,NaN,2015,...,None,[130 minutes],None,2015-08-27,"[huang jue, li jianyi, sean lau, tang wei, qin...","[china, hong kong]",None,None,"[mabel cheung, alex law]",None
